In [1]:
#one-hot编码 单词级
import numpy as np
#samples=['The cat sat on the mat.','The dog ate my homework.','a panda is sleeping.']
samples=['The cat sat on the mat.','The dog ate my homework.']
#构建数据中所有标记的索引，用一个字典来存储
token_index={}
for sample in samples:
    #利用split方法对样本进行分词
    for word in sample.split():
        if word not in token_index:
            #为每个唯一单词指定一个唯一索引
            token_index[word]=len(token_index)+1
            #没有为索引编号0指定单词
print(token_index)

{'The': 1, 'cat': 2, 'sat': 3, 'on': 4, 'the': 5, 'mat.': 6, 'dog': 7, 'ate': 8, 'my': 9, 'homework.': 10}


In [2]:
#对样本进行分词
#只考虑每个样本前max_length个单词
max_length=10
#结果返回给results:
results=np.zeros((len(samples),max_length,max(token_index.values())+1))
for i,sample in enumerate(samples):
    for j,word in list(enumerate(sample.split()))[:max_length]:
        index=token_index.get(word)
        results[i,j,index]=1
#查看索引字典
print(results[1,1])#样本列表的第二个元素第二个单词编码情况

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [3]:
#one-hot编码 字符级
import string
samples=['The cat sat on the mat.','The dog ate my homework.']
characters=string.printable
print(string.printable)
#创建索引字典
token_index=dict(zip(characters,range(1,len(characters)+1)))
#为所有可能打印的字符创建一个字典
max_length=50
results=np.zeros((len(samples),max_length,max(token_index.values())+1))
for i,sample in enumerate(samples):
    for j,character in enumerate(sample[:max_length]):
        index=token_index.get(character)
        results[i,j,index]=1
print(token_index)#查看索引字典
print(results[1,1])#样本列表的第二个元素的第二个字符

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

{'0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36, 'A': 37, 'B': 38, 'C': 39, 'D': 40, 'E': 41, 'F': 42, 'G': 43, 'H': 44, 'I': 45, 'J': 46, 'K': 47, 'L': 48, 'M': 49, 'N': 50, 'O': 51, 'P': 52, 'Q': 53, 'R': 54, 'S': 55, 'T': 56, 'U': 57, 'V': 58, 'W': 59, 'X': 60, 'Y': 61, 'Z': 62, '!': 63, '"': 64, '#': 65, '$': 66, '%': 67, '&': 68, "'": 69, '(': 70, ')': 71, '*': 72, '+': 73, ',': 74, '-': 75, '.': 76, '/': 77, ':': 78, ';': 79, '<': 80, '=': 81, '>': 82, '?': 83, '@': 84, '[': 85, '\\': 86, ']': 87, '^': 88, '_': 89, '`': 90, '{': 91, '|': 92, '}': 93, '~': 94, ' ': 95, '\t': 96, '\n': 97, '\r': 98, '\x0b': 99, '\x0c': 10

In [4]:
#词嵌入模型
from keras.layers import Embedding
from keras.datasets import imdb
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Flatten,Dense
#Embedding层至少需要2个参数
#标记的个数(这里是1000，及最大单词索引+1)和嵌入维度（这里是64）
embedding_layer=Embedding(1000,64)
#作为特征的单词，即选取出现频率最高的单词数量
max_features=10000
#在这么多单词后截断文本
#(这些单词都属于前max_features个最常见单词)
maxlen=20
#将数据集加载为列表
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_features)
#将整数列表转换成形状为(samples,maxlen)的二维整数张量
x_train=preprocessing.sequence.pad_sequences(x_train,maxlen=maxlen)
x_test=preprocessing.sequence.pad_sequences(x_test,maxlen=maxlen)
model=Sequential()
#指定Embedding层的最大输入长度，以便后来将嵌入输入战平
model.add(Embedding(10000,8,input_length=maxlen))
#将三维的嵌入张量展平为(samples,maxlen*8)的二维张量
model.add(Flatten())
#添加分类器
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)
test_loss,test_acc=model.evaluate(x_test,y_test,verbose=1)
print('test_loss:',test_loss,'\n','test_acc:',test_acc)

17465344/17464789 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 1s 2ms/step - loss: 0.6588 - acc: 0.6442 - val_loss: 0.5980 - val_acc: 0.6954
Epoch 2/10
625/625 [==============================] - 1s 1ms/step - loss: 0.5235 - acc: 0.7560 - val_loss: 0.5162 - val_acc: 0.7316
Epoch 3/10
625/625 [==============================] -